# 分桶采样

In [1]:
from utils import *

Done! Loaded utils.py


In [2]:
df_1 = pd.read_csv(f'{ASSETS}labeled_L2__1_technical.csv')
df_2 = pd.read_csv(f'{ASSETS}labeled_L2__2_payment.csv')
df_3 = pd.read_csv(f'{ASSETS}labeled_L2__3_delivery.csv')
df_4 = pd.read_csv(f'{ASSETS}labeled_L2__4_account.csv')
df_5 = pd.read_csv(f'{ASSETS}labeled_L2__5_service.csv')
df_6 = pd.read_csv(f'{ASSETS}labeled_L2__6_pricing.csv')
df_7 = pd.read_csv(f'{ASSETS}labeled_L2__7_legal.csv')

# 计划和实际采样比
# 分类标签样本数
## 计划采样7500
样本中可能含有单一数据被标记多类情况，使用按权重保留策略。

| **Label** | **Bucket**  | **Sampling** | **%**    |
|-----------|-------------|--------------|----------|
| legal     | 5,499       | 220          | 2.9%     |
| account   | 6,060       | 240          | 3.2%     |
| payment   | 39,851      | 1590         | 21.2%    |
| technical | 47,410      | 1890         | 25.2%    |
| delivery  | 60,139      | 2390         | 31.9%    |
| service   | 32,386      | 1290         | 17.2%    |
| pricing   | 2,687       | 105          | 1.4%     |
| **Total** | **194,032** | **7725**     | **100%** |



In [12]:
df_list = [df_1, df_2, df_3, df_4, df_5, df_6, df_7]

sample_sizes = {
    'legal': 220,
    'account': 240,
    'payment': 1590,
    'technical': 1890,
    'delivery': 2390,
    'service': 1290,
    'pricing': 105,
}

# 合并所有数据（并保留原始标签）
df_combined = pd.concat(df_list, ignore_index=True)
df_combined.head()
# 194032

,tweet_id,text,label
0,212,@ATVIAssist @115755 I'm trying to buy the digi...,technical
1,267,@115768 Jeffrey I am sorry to hear that. If yo...,technical
2,323,@1520 @XboxSupport the 5 app on Xboxes not wo...,technical
3,600,@115811 Hi - are you receiving an error messag...,technical
4,1161,@TMobileHelp trying to redeem a free tuesday c...,technical


In [13]:
# 去除重复 tweet_id，只保留优先级高的标签
df_distinct = df_combined.sort_values(
    by='label',
    key=lambda col: col.map(lambda x: RiskLabel[x.upper()].value)
)

df_distinct = df_distinct.drop_duplicates(subset='tweet_id', keep='first')
df_distinct.head()

,tweet_id,text,label
194031,2987073,@118374 Boycotting Argos; reserved item at hal...,legal
190359,1014759,@AmazonHelp Last u are a joke just attract us ...,legal
190360,1014763,@AmazonHelp Public must know your scam,legal
190361,1014770,@AmazonHelp It's a scam they harras us,legal
190362,1015516,@117176 I just received a call from your India...,legal


In [14]:
df_sample = sample_and_clean_enum(df_distinct, sample_sizes)
df_sample.head()
# 7250

,tweet_id,text,label
0,2167038,"#Netflix members, beware: Don’t get tricked by...",legal
1,1696874,"@AppleSupport Just saying, but next time I won...",legal
2,46005,@115714 should fire every last call center wor...,legal
3,2885597,@115783 @AirbnbHelp \nSeems now you don’t have...,legal
4,989846,@AmazonHelp I had excellent exp with amazon in...,legal


In [15]:
# 样本分布，与原始数据集分布保持一致，表现良好
df_sample['label'].value_counts()

label
delivery     2390
technical    1890
payment      1590
service      1290
account       240
legal         220
pricing       105
Name: count, dtype: int64

In [16]:
df_sample.to_csv(f"{ASSETS}labeled_L2__bucket_sampled.csv", index=False, encoding='utf-8-sig')